In [75]:
import gc
import re
from collections import defaultdict
from pprint import pprint

import pandas as pd
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from umap import UMAP

## Датасет

In [76]:
df = pd.read_csv(
    "/content/drive/MyDrive/resume_job_matching/Raw_Jobs.csv", delimiter=";"
)

In [77]:
df.head()

,№,id,title,salary,experience,job_type,description,key_skills,company,location,date_of_post,type
0,1,78577559,Главный механик,от 160000 до 160000 RUR,Более 6 лет,"Полная занятость,полный день","Вакансия компании: ООО ПК Предприятие ""ПИК"" ...",NaN,JCat.ru,Владивосток,27.03.2023,close
1,2,78693069,HTML-верстальщик,з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",В креативное агентство на полный рабочий день ...,NaN,Чугунова Ирина,NaN,29.03.2023,close
2,3,78945452,Тренер (направление профессионального обучения),з/п не указана,От 1 года до 3 лет,"Полная занятость,полный день",Наши предложения: самостоятельность в...,NaN,Перекрёсток,NaN,18.04.2023,close
3,4,79352008,Старший механик на КСПГ,з/п не указана,Более 6 лет,"Полная занятость,полный день",Обязанности: _Обеспечение технически ...,NaN,Газпром гелий сервис,NaN,17.04.2023,close
4,5,79406472,System Analyst Trainee,з/п не указана,Нет опыта,"Стажировка,полный день",IT-компания Aston - компания по разработке про...,"UML,SQL,Scrum,Retail,Базы данных,Kanban,Waterf...",Aston,NaN,17.04.2023,close


Отбираем только IT специальности

In [78]:
valid_comment_pattern = r"программист|разработчик|developer"

In [79]:
df["is_valid_pattern"] = df["title"].apply(
    lambda x: True
    if not isinstance(x, float)
    and re.match(valid_comment_pattern, x.lower()) is not None
    else False
)

In [80]:
df = df[df.is_valid_pattern == True]

Парсим описание вакансии, извлекаем обязанности, требования и условия

In [81]:
responsabilities, req, cond = list(), list(), list()

In [82]:
def extract_information(job_description):
    responsibilities_pattern = r"\b(?:вам предстоит|вы будете|задачи|обязанности|чем необходимо заниматься)\b:(.*?)(?=\b(?:требования|требования к кандидатам|мы предлагаем|условия|требования:|мы ожидаем|наши пожелания|будет плюсом|будет преимуществом)\b|$)"
    requirements_pattern = r"\b(?:для нас важно|ждем от вас|мы ожидаем|наши пожелания|необходимые навыки|требования|требования к кандидатам|будет плюсом|будет преимуществом)\b:(.*?)(?=\b(?:мы предлагаем|условия|место работы|длительность рабочего дня)\b|$)"
    conditions_pattern = r"\b(?:мы предлагаем|условия)\b:(.*?)(?=\b(?:в ответ на оставленный отклик|образование)\b|$)"

    responsibilities_match = re.search(
        responsibilities_pattern, job_description, re.IGNORECASE | re.DOTALL
    )
    requirements_match = re.search(
        requirements_pattern, job_description, re.IGNORECASE | re.DOTALL
    )
    conditions_match = re.search(
        conditions_pattern, job_description, re.IGNORECASE | re.DOTALL
    )

    responsibilities = (
        responsibilities_match.group(1).strip() if responsibilities_match else None
    )
    requirements = requirements_match.group(1).strip() if requirements_match else None
    conditions = conditions_match.group(1).strip() if conditions_match else None

    return responsibilities, requirements, conditions

In [83]:
for row in tqdm(df["description"]):
    responsability, requirements, conditions = extract_information(row)
    responsabilities.append(responsability)
    req.append(requirements)
    cond.append(conditions)

100%|██████████| 1219/1219 [00:00<00:00, 3797.56it/s]


In [84]:
df["responsabilities"] = responsabilities
df["requirements"] = req
df["conditions"] = cond

In [85]:
del responsabilities
del req
del cond
gc.collect()

123394

Делаем текст, который передадим эмбеддеру в bertopic

In [86]:
vacancy_text = defaultdict(str)
for idx, row in df.iterrows():
    vacancy_text[
        idx
    ] = f"Профессия: {row['title']}. Ключевые навыки: {row['key_skills']}. Обязанности: {row['responsabilities']}. Требования: {row['requirements']}. Условия {row['conditions']}"

In [87]:
df["text"] = df.index.map(vacancy_text)

In [88]:
docs = df["text"].tolist()
del df
gc.collect()

0

## BerTopic

Создаем модель BerTopic. В качестве эмбеддера будем использовать multilingual-e5-large, потому что он себя показал наилучшим образом при подсчете метрик

In [89]:
keybert = KeyBERTInspired()
umap_model = UMAP(n_neighbors=15, n_components=10, metric="cosine", low_memory=False)
representation_model = MaximalMarginalRelevance(diversity=0.3)
models = [keybert, representation_model]

embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")
hdbscan_model = HDBSCAN(metric="euclidean", prediction_data=True)
topic_model = BERTopic(
    representation_model=models,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    calculate_probabilities=False,
    low_memory=True,
    top_n_words=25,
    verbose=False,
    nr_topics="auto",
    language="multilingual",
)

topics, probs = topic_model.fit_transform(docs)

In [90]:
topic_model.visualize_topics()

In [113]:
topic_model.get_topics()

{-1: [('программист', 0.8537121),
  ('программирования', 0.84528023),
  ('разработчик', 0.8423854),
  ('программирование', 0.8398908),
  ('разработки', 0.8226284),
  ('требования', 0.82042),
  ('профессия', 0.81645536),
  ('собеседования', 0.81625974),
  ('разработка', 0.81606394),
  ('доработка', 0.81532484)],
 0: [('программист', 0.8485298),
  ('программирования', 0.83769786),
  ('программирование', 0.8301968),
  ('профессия', 0.8223953),
  ('разработчик', 0.8194152),
  ('трудоустройство', 0.8162051),
  ('1с', 0.81517124),
  ('разработке', 0.8108654),
  ('разработки', 0.8084177),
  ('требования', 0.8072736)],
 1: [('программист', 0.8384203),
  ('профессия', 0.82382095),
  ('навыки', 0.8222941),
  ('разработчик', 0.8181265),
  ('требования', 0.8157112),
  ('ключевые', 0.8152171),
  ('python', 0.80785406),
  ('обязанности', 0.80665183),
  ('данных', 0.8042709),
  ('postgresql', 0.80245245)],
 2: [('курсов', 0.82819754),
  ('обучение', 0.82352537),
  ('обучения', 0.8168303),
  ('обучающ

In [91]:
topic_model.save(
    "it_bert_topic",
    serialization="pytorch",
    save_ctfidf=True,
    save_embedding_model=embedding_model,
)

In [114]:
freq = topic_model.get_topic_info()
freq.to_csv("freq.csv", index=False)

## Инференс

In [100]:
loaded_model = BERTopic.load("it_bert_topic", embedding_model=embedding_model)

2023-12-20 22:00:33,626 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model.If you want to also load in an embedding model, make sure to useBERTopic.load(my_model, embedding_model=my_embedding_model).


In [105]:
query = """Программист C++, Web-программист, разработчик аппаратуры,.Опыт работы 5 лет 8 месяцев  Март 2019 — по настоящее время 3 месяца Удаленная работа Обнинск Web-программист Доработка сайта www.autolarek24.ru  Октябрь 2013 — по настоящее время 5 лет 8 месяцев АО «ГНЦ РФ – ФЭИ» Обнинск , www.ippe.ru Энергетика ... Атомная энергетика (генерация электроэнергии, АЭС) Инженер-программист Разработка ПО для операционных систем жесткого реального времени. Разработка драйверов, многопоточных приложений, клиент-серверных приложений. Разработка микропроцессорных систем. Работа с промышленными интерфейсами и протоколами: ModBus, CAN, RS422/485. Разработка документации ПО. Работа с измерительной аппаратурой с очень высокой точностью измерения и написание ПО для мониторинга и фильтрации измерений, а так же участие в разработке, настройке и тестировании данной аппаратуры.  Июнь 2017 — Ноябрь  2017 6 месяцев ООО "Персона" Обнинск Информационные технологии, системная интеграция, интернет ... Разработка программного обеспечения Веб-разработчик Разработка веб-сайтов, концепция MVC, командная разработка (Git), Laravel. Проект www.combeep.com"""

In [106]:
topics, predicts = loaded_model.transform(query)

Достанем документы

In [116]:
docs = pd.read_csv("freq.csv")
docs.head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,271,-1_программист_программирования_разработчик_пр...,"['программист', 'программирования', 'разработч...",['Профессия: Программист/Разработчик C#/.NET. ...
1,0,639,0_программист_программирования_программировани...,"['программист', 'программирования', 'программи...",['Профессия: Разработчик 1С (удаленно). Ключев...
2,1,50,1_программист_профессия_навыки_разработчик,"['программист', 'профессия', 'навыки', 'разраб...",['Профессия: Разработчик Python. Ключевые навы...
3,2,49,2_курсов_обучение_обучения_обучающих,"['курсов', 'обучение', 'обучения', 'обучающих'...",['Профессия: Разработчик электронных курсов Is...
4,3,45,3_программист_программ_фрезерные_фрезерных,"['программист', 'программ', 'фрезерные', 'фрез...",['Профессия: Программист станков с ЧПУ. Ключев...


In [123]:
pprint(docs[docs["Topic"] == topics[0]])

   Topic  Count                                               Name  \
1      0    639  0_программист_программирования_программировани...   

                                      Representation  \
1  ['программист', 'программирования', 'программи...   

                                 Representative_Docs  
1  ['Профессия: Разработчик 1С (удаленно). Ключев...  


## Вывод

Качество поиска нас не устраивает, будем пробовать другие подходы